<a href="https://colab.research.google.com/github/Mahfujul-01726/Random/blob/main/SynergyX2024_Datathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
data.shape

(621165, 68)

In [ ]:
# Count NaN values in the target column 'v16'
nan_count = final_df['v16'].isna().sum()

print(f"Number of NaN values in 'v16': {nan_count}")

Number of NaN values in 'v16': 0


In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier  # Use RandomForestRegressor for regression tasks
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report  # For classification tasks
from sklearn.impute import SimpleImputer  # For imputing missing target values

# Load the dataset
data = pd.read_csv("train.csv")

# Separate features and target
X = data.drop(columns=['v16'], errors='ignore')  # Feature matrix without target column
y = data['v16'] if 'v16' in data.columns else None  # Target variable

# Drop non-numeric columns from features, assuming 'id' is retained separately for later merging
X_numeric = X.select_dtypes(include=[int, float])

# Impute missing values in the feature matrix using the mean (for numeric features)
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X_numeric)

# Initialize the SimpleImputer for the target variable 'v16' (impute with the most frequent class)
y_imputer = SimpleImputer(strategy='most_frequent')  # For classification, you can also use 'mean' for regression
y_imputed = y_imputer.fit_transform(data[['v16']])

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_imputed, y_imputed.ravel(), test_size=0.2, random_state=42)

# Initialize the Random Forest Classifier (use RandomForestRegressor if it's a regression task)
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)  # Adjust parameters as needed

# Train the model
rf_model.fit(X_train, y_train)

# Predict on the test set
y_pred = rf_model.predict(X_test)

# Evaluate the model (for classification)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))

# If it's a regression problem, you can use the following metrics instead:
# from sklearn.metrics import mean_squared_error, r2_score
# print("Mean Squared Error:", mean_squared_error(y_test, y_pred))
# print("R^2 Score:", r2_score(y_test, y_pred))


Accuracy: 0.9772685196364895
Classification Report:
              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99    121237
         1.0       0.89      0.07      0.12      2996

    accuracy                           0.98    124233
   macro avg       0.93      0.53      0.56    124233
weighted avg       0.98      0.98      0.97    124233



# Suppose it is final code

In [ ]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier  # Use RandomForestRegressor if regression task
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report  # For classification tasks
# from sklearn.metrics import mean_squared_error, r2_score  # Uncomment for regression tasks

# Load the dataset
data = pd.read_csv("train.csv")

# Separate features and target
X = data.drop(columns=['v16'], errors='ignore')  # Feature matrix without target column
y = data['v16'] if 'v16' in data.columns else None  # Target variable

# Drop non-numeric columns from features, assuming 'id' is retained separately for later merging
X_numeric = X.select_dtypes(include=[int, float])

# Impute missing values in the feature matrix using the mean
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X_numeric)

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

# Apply PCA to retain 95% of the variance
pca = PCA(0.95)
X_pca = pca.fit_transform(X_scaled)

# Create a DataFrame with the principal components
pca_columns = [f'PC{i+1}' for i in range(X_pca.shape[1])]
X_pca_df = pd.DataFrame(data=X_pca, columns=pca_columns)

# Combine with the original 'id' column if it exists
if 'id' in data.columns:
    final_df = pd.concat([data[['id']].reset_index(drop=True), X_pca_df], axis=1)
else:
    print("Warning: 'id' column not found in the original data. Proceeding without it.")
    final_df = X_pca_df

# Handle missing values in the target variable 'v16' using imputation
if y is not None:
    target_imputer = SimpleImputer(strategy='most_frequent')  # Impute with the most frequent class for classification
    y_imputed = target_imputer.fit_transform(y.values.reshape(-1, 1))  # Reshaping for imputation

    # Add the imputed target variable back to the final DataFrame
    final_df = pd.concat([final_df, pd.Series(y_imputed.flatten(), name='v16').reset_index(drop=True)], axis=1)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(final_df.drop(columns=['v16'], errors='ignore'),
                                                    final_df['v16'],
                                                    test_size=0.2,
                                                    random_state=42)

# Initialize the Random Forest Classifier (use RandomForestRegressor for regression tasks)
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)  # You can adjust the parameters

# Train the model
rf_model.fit(X_train, y_train)

# Predict on the test set
y_pred = rf_model.predict(X_test)

# Evaluate the model (for classification)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))

# If it's a regression problem, you can use the following metrics instead:
# from sklearn.metrics import mean_squared_error, r2_score
# print("Mean Squared Error:", mean_squared_error(y_test, y_pred))
# print("R^2 Score:", r2_score(y_test, y_pred))


Accuracy: 0.9758759749824926
Classification Report:
              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99    121237
         1.0       0.47      0.00      0.01      2996

    accuracy                           0.98    124233
   macro avg       0.72      0.50      0.50    124233
weighted avg       0.96      0.98      0.96    124233



# importing dataset and merge

In [15]:
import pandas as pd

# Load the datasets
df1 = pd.read_csv('train.csv')
df2 = pd.read_csv('test.csv')

# Define the columns to merge on, excluding 'v16'
merge_columns = ['id'] + [f'v{i}' for i in range(1, 68) if i != 16]  # Excludes 'v16'

# Step 1: Merge DataFrames based on specified columns (merging columns)
merged_columns_df = pd.merge(df1, df2, on=merge_columns, how='inner')  # Adjust 'how' as needed (inner, outer, left, right)

# Step 2: Concatenate rows (row-wise merge)
merged_rows_columns_df = pd.concat([merged_columns_df, df1, df2], axis=0, ignore_index=True).drop_duplicates()

# View the final merged DataFrame
merged_rows_columns_df.head()


,id,v1,v2,v3,v4,v5,v6,v7,v8,v9,...,v58,v59,v60,v61,v62,v63,v64,v65,v66,v67
0,1,36802507,39555223,10000.0,10000.0,10000.0,0,12.39,334.01,2,...,NaN,NaN,NaN,NaN,NaN,NaN,8400.0,NaN,NaN,NaN
1,2,34373063,37036411,20000.0,20000.0,20000.0,1,16.49,491.59,3,...,NaN,NaN,NaN,NaN,NaN,NaN,12100.0,NaN,NaN,NaN
2,3,39329649,42133381,8000.0,8000.0,8000.0,0,9.49,256.23,1,...,NaN,NaN,NaN,NaN,NaN,NaN,16000.0,NaN,NaN,NaN
3,4,54898516,58469236,16000.0,16000.0,16000.0,0,11.53,527.85,1,...,NaN,NaN,NaN,NaN,NaN,NaN,8300.0,NaN,NaN,NaN
4,5,42513663,45480375,10200.0,10200.0,10200.0,0,18.25,370.04,4,...,NaN,NaN,NaN,NaN,NaN,NaN,19600.0,NaN,NaN,NaN


In [16]:
merged_rows_columns_df.shape

(807899, 68)

In [17]:
print(merged_rows_columns_df)

            id        v1        v2       v3       v4       v5  v6     v7  \
0            1  36802507  39555223  10000.0  10000.0  10000.0   0  12.39   
1            2  34373063  37036411  20000.0  20000.0  20000.0   1  16.49   
2            3  39329649  42133381   8000.0   8000.0   8000.0   0   9.49   
3            4  54898516  58469236  16000.0  16000.0  16000.0   0  11.53   
4            5  42513663  45480375  10200.0  10200.0  10200.0   0  18.25   
...        ...       ...       ...      ...      ...      ...  ..    ...   
807894  186730  22131285  24454239  10800.0  10800.0  10800.0   1  16.99   
807895  186731  14570817  16633101  10725.0  10725.0  10725.0   0  17.57   
807896  186732  34413316  37076669  16000.0  16000.0  16000.0   0   8.19   
807897  186733  64997380  69524101  18000.0  18000.0  18000.0   1  12.59   
807898  186734  56082809  59744686  23750.0  23750.0  23750.0   1  26.77   

            v8  v9  ...  v58  v59  v60  v61  v62 v63      v64  v65  v66  v67  
0       

# this is good approach for v16 value Predictive Imputation with a Classification Model

In [20]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier  # Use RandomForestClassifier for binary imputation
from sklearn.impute import SimpleImputer

# Load the dataset
data = merged_rows_columns_df

# Separate features and target
X = data.drop(columns=['v16'], errors='ignore')  # Feature matrix without target column
y = data['v16'] if 'v16' in data.columns else None  # Target variable

# Identify non-numeric columns
non_numeric_cols = X.select_dtypes(exclude=[int, float]).columns
print("Non-numeric columns:", non_numeric_cols)

# Apply Label Encoding for non-numeric columns (Option 1: ordinal data)
label_encoder = LabelEncoder()
for col in non_numeric_cols:
    X[col] = label_encoder.fit_transform(X[col])

# Optionally, you could use one-hot encoding instead if appropriate for the data type
# X = pd.get_dummies(X, columns=non_numeric_cols, drop_first=True)

# Drop non-numeric columns from features (now they are encoded)
X_numeric = X.select_dtypes(include=[int, float])

# Impute missing values in the feature matrix using the mean
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X_numeric)

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

# Apply PCA to retain 95% of the variance
pca = PCA(0.95)
X_pca = pca.fit_transform(X_scaled)

# Create a DataFrame with the principal components
pca_columns = [f'PC{i+1}' for i in range(X_pca.shape[1])]
X_pca_df = pd.DataFrame(data=X_pca, columns=pca_columns)

# Combine with the original 'id' column if it exists
if 'id' in data.columns:
    final_df = pd.concat([data[['id']].reset_index(drop=True), X_pca_df], axis=1)
else:
    print("Warning: 'id' column not found in the original data. Proceeding without it.")
    final_df = X_pca_df

# Predictive Imputation for the target variable 'v16' using a classifier
if y is not None:
    # Separate rows with and without missing values in v16
    X_no_nan = X_pca_df[y.notna()].reset_index(drop=True)  # Rows without missing v16
    y_no_nan = y.dropna().reset_index(drop=True)           # Target values without NaN
    X_nan = X_pca_df[y.isna()].reset_index(drop=True)      # Rows with missing v16

    # Train a RandomForestClassifier to predict missing values in v16
    classifier = RandomForestClassifier()
    classifier.fit(X_no_nan, y_no_nan)

    # Predict missing values in v16
    y_pred_nan = classifier.predict(X_nan)

    # Fill missing values with the predicted values
    y_imputed = y.copy()
    y_imputed.loc[y.isna()] = y_pred_nan

    # Add the imputed target variable back to the final DataFrame
    final_df = pd.concat([final_df, pd.Series(y_imputed, name='v16').reset_index(drop=True)], axis=1)

print(final_df)


Non-numeric columns: Index(['v15', 'v39', 'v41', 'v42'], dtype='object')
            id       PC1       PC2       PC3       PC4       PC5       PC6  \
0            1 -1.906399 -0.230438  0.554499  0.864747 -0.163991 -2.054459   
1            2  1.441092  1.595874  0.971041  1.719461 -0.306974 -0.222587   
2            3 -2.778585 -0.133561 -0.266440 -0.115158 -0.303129 -2.555043   
3            4 -1.163664  2.118228 -0.133977 -0.072008  0.130287  1.138558   
4            5 -1.408326  0.480652  1.428213  2.021121 -0.133214  0.299666   
...        ...       ...       ...       ...       ...       ...       ...   
807894  186730 -0.673428  0.183702  1.538047  1.909149  0.195424  0.249856   
807895  186731 -0.332331 -0.665859  0.753908  1.037999  0.092318 -0.101186   
807896  186732 -0.047101  0.896069 -1.174028 -1.820856  0.158890 -0.126321   
807897  186733 -0.107809  3.023788  0.514548  0.650291  0.061750 -0.261612   
807898  186734  3.026031  3.281989  2.683374  3.515061  0.323666  0.8

In [24]:
import pandas as pd

# Assuming your DataFrame is named df and the column is 'v16'
count_0_1 = final_df['v16'].value_counts()

print(count_0_1)


v16
0.0    751025
1.0     56874
Name: count, dtype: int64


# Used random forest algorithm

In [21]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import joblib  # For saving the model


# Now, train and evaluate a Random Forest model on the entire dataset

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_pca_df, y_imputed, test_size=0.2, random_state=42)

# Initialize the RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Predict on the test set
y_pred = rf_model.predict(X_test)

# Evaluate the model (for classification)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.9832900111399926
Classification Report:
              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99    150259
         1.0       0.95      0.80      0.87     11321

    accuracy                           0.98    161580
   macro avg       0.97      0.90      0.93    161580
weighted avg       0.98      0.98      0.98    161580



# Apply Logistic regression algorithm

In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_pca_df, y_imputed, test_size=0.2, random_state=42)

# Initialize the Logistic Regression model
logreg_model = LogisticRegression(random_state=42)

# Train the model
logreg_model.fit(X_train, y_train)

# Predict on the test set
y_pred = logreg_model.predict(X_test)

# Evaluate the model (for classification)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 0.9637517019433098
Classification Report:
              precision    recall  f1-score   support

         0.0       0.97      0.99      0.98    150259
         1.0       0.85      0.58      0.69     11321

    accuracy                           0.96    161580
   macro avg       0.91      0.79      0.84    161580
weighted avg       0.96      0.96      0.96    161580



# Apply K-Nearest Neighbors (KNN)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_pca_df, y_imputed, test_size=0.2, random_state=42)

# Initialize the KNeighborsClassifier model
knn_model = KNeighborsClassifier(n_neighbors=5)  # You can experiment with different values for n_neighbors

# Train the model
knn_model.fit(X_train, y_train)

# Predict on the test set
y_pred = knn_model.predict(X_test)

# Evaluate the model (for classification)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))


# Apply  Support Vector Machine (SVM) algorithm

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_pca_df, y_imputed, test_size=0.2, random_state=42)

# Initialize the Support Vector Classifier model
svm_model = SVC(random_state=42)

# Train the model
svm_model.fit(X_train, y_train)

# Predict on the test set
y_pred = svm_model.predict(X_test)

# Evaluate the model (for classification)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))
